In [1]:
experiment_name = "news_exp"

In [2]:
initial_prompt = """
You goal is to classify the following news headlines into one of the following categories:
1- World
2- Sports
3- Business
4- Sci/Tech

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output just one single category."
}
"""

In [3]:
# Output format prompt
output_format_prompt = """
Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output just one single category."
}
"""

In [4]:
# Define output schema
output_schema = {
    'key_to_extract': 'topic',
    'value_mapping': {
        'World': 'World',
        'Sports': 'Sports',
        'Business': 'Business',
        'Sci/Tech': 'Sci/Tech',
    },
    'regex_pattern': r'"topic":\s*"([^"]+)"',  # Changed from \w+ to [^"]+ to capture everything until the closing quote
    #
    'chain_of_thought_key': 'chain_of_thought',  
    'chain_of_thought_regex': r'"chain_of_thought":\s*"(.*?)"',
    #
    'use_json_mode': True,
}

In [5]:
# Set number of optimization iterations
iterations = 3

In [6]:
# Define model providers and models for evaluation and optimization
eval_provider = "openai"
eval_model = "gpt-4o-mini"
optim_provider = "openai"
optim_model = "gpt-4o-mini"

In [7]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "news.csv"
text_column = "Title"
target_column = "topic"
sample_size = 5

------------------------------------------------------------------------------------------

In [8]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)
from src import optimize_prompt

In [9]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=[text_column, target_column])
eval_data = eval_data.rename(columns={text_column: 'text', target_column: 'label'})
eval_data["text"] = eval_data["text"].astype(str)
eval_data

,text,label
0,Fears for T N pension after talks,Business
1,The Race is On: Second Private Team Sets Launc...,Sci/Tech
2,Ky. Company Wins Grant to Study Peptides (AP),Sci/Tech
3,Prediction Unit Helps Forecast Wildfires (AP),Sci/Tech
4,Calif. Aims to Limit Farm-Related Smog (AP),Sci/Tech
...,...,...
7595,Around the world,World
7596,Void is filled with Clement,Sports
7597,Martinez leaves bitter,Sports
7598,5 of arthritis patients in Singapore take Bext...,Business


In [10]:
# Randomly select n samples from each class
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=sample_size, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (20, 2)
                                                text     label
0  Ford: Monthly Sales Drop, Company Looks To New...  Business
1          Scores of Iraqis die in 3 days of attacks     World
2  Large Explosion Heard in Central Baghdad (Reut...     World
3                       United #39;s pension dilemma  Business
4                    Media center at your fingertips  Sci/Tech


In [11]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt = initial_prompt,
    eval_data = eval_data,
    iterations =iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    eval_temperature=0.7,
    optim_provider=optim_provider,
    optim_model=optim_model,
    optim_temperature=0,
    use_cache=True,
    output_format_prompt = output_format_prompt,
    output_schema=output_schema,
    fp_comments = "",
    fn_comments = "",
    tp_comments = "",
    invalid_comments="",
    validation_comments="",
    experiment_name = experiment_name,
    skip_prompt_validation = True,
)


Detected problem type: multiclass
Selected evaluation provider: openai
Selected evaluation model: gpt-4o-mini
Evaluation temperature: 0.7
Selected optimization provider: openai
Selected optimization model: gpt-4o-mini
Optimization temperature: 0
Estimated token usage: 27360
Estimated cost: $0.03

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│                                                                                                                 │
│ You goal is to classify the following news headlines into one of the following categories:                      │
│ 1- World                                                                                                        │
│ 2- Sports                                                                                                       │
│ 3- Business                                                                                                     │
│ 4- Sci/Tech                                                                                                     │
│                                                                                                                 │
│ Provide your response as a JSON dictionary with the following structure:                                        │
│ {                                                                                                               │
│     "chain_of_thought": "Your step-by-step reasoning here",                                                     │
│     "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech".    │
│ Output just one single category."                                                                               │
│ }                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

-----------------------------------
Processing text 1/20 .....
Using cached output for text 1/20
Prediction 1/20: Business | Ground Truth: Business ✅ (Correct)
-----------------------------------
Processing text 2/20 .....
Using cached output for text 2/20
Prediction 2/20: World | Ground Truth: World ✅ (Correct)
-----------------------------------
Processing text 3/20 .....
Using cached output for text 3/20
Prediction 3/20: World | Ground Truth: World ✅ (Correct)
-----------------------------------
Processing text 4/20 .....
Using cached output for text 4/20
Prediction 4/20: Business | Ground Truth: Business ✅ (Correct)
-----------------------------------
Processing text 5/20 .....
Using cached output for text 5/20
Prediction 5/20: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Processing text 6/20 .....
Using cached output for text 6/20
Prediction 6/20: World | Ground Truth: Sci/Tech ❌ (Incorrect)
-----------------------------------
Processing text 7

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 0.9583 │
│ Recall              │ 0.9500 │
│ Accuracy            │ 0.9500 │
│ F1-score            │ 0.9495 │
│                     │        │
├─────────────────────┼────────┤
│ Valid Predictions   │     20 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘

       Confusion Matrix        
┏━━━━━━━━━━━━━┳━━━┳━━━┳━━━┳━━━┓
┃ True↓/Pred→ ┃ 0 ┃ 1 ┃ 2 ┃ 3 ┃
┡━━━━━━━━━━━━━╇━━━╇━━━╇━━━╇━━━┩
│ 0           │ 5 │ 0 │ 0 │ 0 │
├─────────────┼───┼───┼───┼───┤
│ 1           │ 0 │ 4 │ 0 │ 1 │
├─────────────┼───┼───┼───┼───┤
│ 2           │ 0 │ 0 │ 5 │ 0 │
├─────────────┼───┼───┼───┼───┤
│ 3           │ 0 │ 0 │ 0 │ 5 │
└─────────────┴───┴───┴───┴───┘

...

## Improved Prompt:

Your goal is to classify the following news headlines into one of the following categories:
1- World
2- Sports
3- Business
4- Sci/Tech

When classifying, consider the following guidelines:
- **World**: Focus on international relations, conflicts, and significant global events. Look for keywords related 
to countries and geopolitical issues.
- **Sports**: Identify mentions of specific sports, athletes, or competitions.
- **Business**: Look for financial terms, corporate actions, and market-related language. Be cautious of headlines 
that may involve technology but are primarily about economic activities.
- **Sci/Tech**: Focus on technological advancements, scientific discussions, and regulatory actions related to 
technology. Be aware that headlines involving government actions may also relate to World news.

Avoid misclassifying headlines that blend societal issues with technology. If a headline discusses regulations 
affecting technology, prioritize the technological context over political implications.

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here, ensuring to balance political and technological contexts
and clarify category boundaries.",
    "topic": "The identified topic - the ONLY possible topics are 'World', 'Sports', 'Business', 'Sci/Tech'. Output
just one single category."
}

╭────────────────╮
│ Output Format: │
╰────────────────╯

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output
just one single category."
}

                                   Comparison of All Iterations                                   
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Iteration ┃ Precision ┃ Recall ┃ Accuracy ┃ F1-score ┃ Valid Predictions ┃ Invalid Predictions ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│     1     │    0.9583 │ 0.9500 │   0.9500 │   0.9495 │           20.0000 │              0.0000 │
│     2     │    1.0000 │ 1.0000 │   1.0000 │   1.0000 │           19.0000 │              1.0000 │
│     3     │    1.0000 │ 1.0000 │   1.0000 │   1.0000 │           20.0000 │              0.0000 │
└───────────┴───────────┴────────┴──────────┴──────────┴───────────────────┴─────────────────────┘


All logs saved in directory: /Users/danielfiuzadosil/Documents/GitHub/AI-Prompt-Optimiser/examples/muticlass_classifier/runs/news_exp_Tue_29-Oct-2024_10-42-30
